On trouve la formule grâce aux remarques suivantes

On note $C(n)$ le nombre de cycles dans un triangle de Sierpinski d'odre n. En notant P(n) le nombre de chemins hamiltoniens qui partent du côté A d'un triangle de rang n pour finir au sommet B, on remarque assez simplement :

$$C(n) = P(n-1)^3$$

Par ailleurs, en notant $K(n)$ le nombre de chemins qui partent d'un sommet A pour arriver au sommet B en passant pour tous les points *sauf* le sommet C, on trouve :

$$P(n+1) = 2P(n)^2K(n)$$
$$K(n+1) = 2P(n)K(n)^2$$

à partir de n = 2 (pour n = 1, la deuxième relation n'est pas vérifiée)

On constate donc :

$$P(n+1)K(n+1) = 4(P(n)K(n))^2$$

Avec $P(2) = 2$ et $K(2) = 3$

On en déduit que $P(n)K(n) = 4^{u_n} 6^{v_n}$

Au final:

$$C(n) = 8 \times 12^{(3^{n-2}-3)/ 2}$$

In [1]:
def C_naif(n):
    return 8 * pow(12, (3**(n-2) - 3) // 2)

On devrait avoir $C(5) = 71328803586048$

In [2]:
C_naif(5) == 71328803586048

True

Il faut calculer C(C(C(10000)))

In [10]:
from sympy.ntheory import factorint
from functools import reduce
import operator as op

def recomp(decomp):
    """Transforme une représentation en décomposition en nombres premiers en un entier
    """
    return reduce(op.mul, (p**k for p, k in decomp.items()))

def phi(n):
    """Renvoie la valeur de l'indicatrice d'Euler comme une décomposition en nombres premiers
    """
    if isinstance(n, int):
        n = factorint(n)
    # n must be a dict !
    
    res = {}
    
    for p, k in n.items():
        new_factors = factorint(p-1)
        for np, nk in new_factors.items():
            res[np] = nk + res.get(np, 0)
        if k != 1:
            res[p] = k-1 + res.get(p, 0)
        
    return res

def find_order(elem, mod):
    """Renvoie l'ordre d'un élément du groupe des entiers modulos mod inversibles
    
    Attention, elem doit être premier avec mod, ou cette fonction lève une ValueError
    """
    if isinstance(mod, int):
        mod_int = mod
        mod = factorint(mod)
    else:
        mod_int = recomp(mod)
        
    tot = phi(mod)
    
    res = tot.copy()
    
    for p, e in tot.items():
        res[p] = 0
        while pow(elem, recomp(res), mod_int) != 1 and res[p] <= tot[p]:
            res[p] += 1
        
        if res[p] > tot[p]: raise ValueError("elem n'est pas premier avec mod!")
        if res[p] == 0: del res[p]
    
    return res

def remove_common(a, b):
    """Enlève dans a tous les nombres premiers présents dans b
    
    Cette fonction renvoie un nouveau dictionnaire et ne touche pas à ses arguments
    
    Par exemple, si a = {2: 4, 3: 2, 13: 4} et b = {2:1, 13:6}, alors
    cette fonction renvoie {3: 2}
    """
    res = a.copy()
    for p in b:
        if p in a:
            del res[p]
    return res

def modi(a, mod):
    """Modulo légèrement modifié
    
    Ce modulo renvoie un nombre entre 1 et mod (compris).
    Le modulo classique renvoie de 0 à mod-1.
    """
    return ((a-1) % mod) + 1

def powi(a, b, c):
    """Exponentiation modulaire légèrement modifiée
    
    La fonction modi est appliquée au résultat final.
    """
    return modi(pow(a, b, c), c)

def gen_c_mod(mod):
    """Génère une fonction de comptage des cycles sierpinski modulo mod
    
    Renvoie deux arguments : la fonction générée, et le modulo qui s'applique
    sur l'argument (pour pouvoir construire les étages supérieurs).
    """
    # si mod est un entier (plutôt qu'une décomposition en nombres premiers)
    # on le décompose d'abors
    if isinstance(mod, int):
        mod = factorint(mod)
    mod_over_12 = find_order(12, remove_common(mod, factorint(12)))
    
    # as the 3 to the power is divided by two, we must multiply that mod by 2
    mod_over_12[2] = 1 + mod_over_12.get(2, 0)
    
    mod_over_3 = find_order(3, remove_common(mod_over_12, factorint(3)))
    
    m12 = recomp(mod_over_12)
    m3 = recomp(mod_over_3)
    m = recomp(mod)
    
    def f(n):
        nm2 = modi(n-2, m3)
        k = powi(3, nm2, m12)
        return modi(8*powi(12, (k-3)//2, m), m)
    
    return f, mod_over_3

On teste avec les valeurs données:

$$C(10000) mod 10^8 = 37652224$$
$$C(10000) mod 13^8 = 617720485$$

In [11]:
gen_c_mod(10**8)[0](10000) == 37652224

True

In [12]:
gen_c_mod(13**8)[0](10000) == 617720485

True

On obtient le résultat en générant les trois fonctions partielles.

In [13]:
m0 = 13**8
c1, m1 = gen_c_mod(m0)
c2, m2 = gen_c_mod(m1)
c3, m3 = gen_c_mod(m2)
c1(c2(c3(10000)))

324681947

Pour mémoire, une implémentation du logarithme discret

In [8]:
from sympy.ntheory.modular import crt

def discrete_log(g, mod, e):
    # find x so that base**x % mod == g
    # mod should be in dict form
    
    totient = phi(mod)
    tot = recomp(totient)
    g_inv = mulinv(g, mod)
    
    new_mods = []
    congs = []
    
    for p, k in totient.items():
        new_tot = tot
        
        partial_powers = []
        
        e_cur = e
        g_pow = pow(g, new_tot//p, mod)
        
        for i in range(1,k+1):
            new_tot //= p
            e_pow = pow(e_cur, new_tot, mod)
            if g_pow == 1 and e_pow != 1:
                raise ValueError("pas de solution !")
            
            for j in range(p):
                if pow(g_pow, j, mod) == e_pow:
                    break
            
            partial_powers.append(j)
            
            if i != k:
                e_cur = e_cur * pow(g_inv, p**(i-1)*j, mod) % mod
                
        new_mods.append(p**k)
        congs.append(sum(j * p**(i) for i, j in enumerate(partial_powers)))
    
    return crt(new_mods, congs)[0]